In [1]:
import os

os.chdir('../')

print(os.getcwd())

/Users/gbemidebe/Documents/LSUCourses/Fall2023/CSC7343/homeworks/Project


In [12]:
import torch
import torch.nn as nn

import random
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

## Read Data

In [14]:
data = pd.read_csv('data//data.csv')
data.head()

,antigen,TCR,interaction
0,AAGIGILTV,CAISEVGVGQPQHF,1
1,AAGIGILTV,CASSLSFGTEAFF,1
2,AARAVFLAL,CASLGAQNNEQF,1
3,AARAVFLAL,CASSYSTGDEQYF,1
4,AIMDKNIIL,CASSVDGGSQPQHF,1


In [17]:
data[['antigen', 'TCR']].to_csv('data/data_no_interation.csv', index=False, header=False)

In [3]:
# fix random seeds
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
    
# load CSV dataset
antigen = []
TCR = []
interaction=[]
with open("./data/data.csv") as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    next(csvReader)
    for row in csvReader:
        antigen.append(row[0])
        TCR.append(row[1])
        interaction.append(row[2])

antigen = np.asarray(antigen, dtype='str')
TCR = np.asarray(TCR, dtype='str')
interaction = np.asarray(interaction, dtype="int")


dataset_size = len(antigen)
print("Dataset size: ", dataset_size)
all_ind = np.arange(dataset_size)

Dataset size:  130471


In [7]:
len(set(antigen)), len(set(TCR))

(400, 127277)

In [9]:
def find_unique_characters(str_list):
    # Initialize an empty set to store unique characters
    unique_chars = set()

    # Iterate over each string and update the set with unique characters from that string
    for s in str_list:
        unique_chars.update(set(s))

    # Convert the set to a sorted string
    return ''.join(sorted(unique_chars))

# Example usage
str_array = ["hello", "world", "owl", "teslim"]
print(sorted(find_unique_characters(str_array)))


['d', 'e', 'h', 'i', 'l', 'm', 'o', 'r', 's', 't', 'w']


## Prepare input

In [10]:
# get unique elements in antigen and TCR
unique_antigen = sorted(find_unique_characters(antigen))
unique_TCR = sorted(find_unique_characters(TCR))
print("Unique antigen: ", unique_antigen, len(unique_antigen))
print("Unique TCR: ", unique_TCR, len(unique_TCR))

Unique antigen:  ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] 20
Unique TCR:  ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] 20


In [6]:
# Determine the maximum lengths of antigen and TCR sequences
max_antigen_length = max(len(item[0]) for item in antigen)
max_tcr_length = max(len(item[1]) for item in TCR)

# Function to pad sequences to a given length
def pad_sequence(sequence, max_length):
    return sequence + 'X' * (max_length - len(sequence))  # Using 'X' as the padding character

# Preprocess and create the combined sequences
preprocessed_data = []
for antigen, tcr in zip(antigen, TCR):
    padded_antigen = pad_sequence(antigen, max_antigen_length)
    padded_tcr = pad_sequence(tcr, max_tcr_length)
    combined_sequence = f"<cls>{padded_antigen}<sep>{padded_tcr}"
    preprocessed_data.append(combined_sequence)

# Displaying the first few combined sequences for verification
preprocessed_data[:5]  # Display the first 5 combined sequences


['<cls>AAGIGILTV<sep>CAISEVGVGQPQHF',
 '<cls>AAGIGILTV<sep>CASSLSFGTEAFF',
 '<cls>AARAVFLAL<sep>CASLGAQNNEQF',
 '<cls>AARAVFLAL<sep>CASSYSTGDEQYF',
 '<cls>AIMDKNIIL<sep>CASSVDGGSQPQHF']

In [7]:
max_antigen_length + max_tcr_length

2

In [8]:
preprocessed_data[5:100]

['<cls>ALGIGILTV<sep>CASSFLGTGVEQYF',
 '<cls>ALIHHNTHL<sep>CASSIESGSKQRNEQFF',
 '<cls>ALIHHNTHL<sep>CASSLAVQRPSGNTIYF',
 '<cls>ALIHHNTHL<sep>CASSPRDRVHEQYF',
 '<cls>ALIHHNTHL<sep>CASSQSETGDGYTF',
 '<cls>ALIHHNTYL<sep>CASSDLQENQPQHF',
 '<cls>ALIHHNTYL<sep>CSVVGTGSVITNEKLFF',
 '<cls>ALLETLSLLL<sep>CSARNRDTYYNEQFF',
 '<cls>ALLQVTLLL<sep>CASSSTDGYQETQYF',
 '<cls>ALSPVIPHI<sep>CASSFSGTFEAFF',
 '<cls>ALSPVIPHI<sep>CASSPVLGTYGYTF',
 '<cls>ALSPVIPHI<sep>CASSYFLEQYF',
 '<cls>ALSPVIPLI<sep>CAISETGGGQPQHF',
 '<cls>ALSPVIPLI<sep>CASSGAAGTSAYNEQFF',
 '<cls>ALSPVIPLI<sep>CASSGQGAYITDTQYF',
 '<cls>ALSPVIPLI<sep>CASSLGTLNEQFF',
 '<cls>ALSPVIPLI<sep>CASSPVLGTYGYTF',
 '<cls>ALSPVIPLI<sep>CASSQGPPGSGGGNEQFF',
 '<cls>ALSPVIPLI<sep>CASSVETGGLDTQYF',
 '<cls>ALSPVIPLI<sep>CASSYSSDRSSYEQYF',
 '<cls>ALSPVIPLI<sep>CASYPISRASYEQYF',
 '<cls>ALSPVIPLI<sep>CSATCNGTFLYQETQYF',
 '<cls>ALSYTPAEV<sep>CASSLGQEIPYYGYTF',
 '<cls>ALSYTPAEV<sep>CASSSPSGYEQYF',
 '<cls>ALSYTPAEV<sep>CASSTQGTGVYNEQFF',
 '<cls>ALTPVVVTL<sep>CAS